## 摄像头FTP设置

1. 参数设置 > 网络设置 > RTMP Publish > 获取mac地址

![](./images/cam-1.png)

------------

2. 参数设置 > 高级设置 > FTP设置 (注意mac地址必须和上一步获取的一致, 设置后点击测试看一下是否已通)

> 密码（暂时）: holyhand 

![](./images/cam-2.png)

--------------

3. 参数设置 > 定时抓拍 > FTP时间间隔 （具体设置的值, 需要根据不同的场景确定）

![](./images/cam-3.png)

## 结构图

```txt

COS_DOMAIN: htpps://cosftpd-1301930378.cos.ap-beijing.myqcloud.com

    ╭───────────╮
    │           │                  ${COS_DOMAIN}/holyhand/008e9b61ac6c/20221104/images/P22110420294710.jpg
    │  Camera   │                                        ╭─────────────╮
    │    +ftp   │                              put       │             │
    │           │                       ───────────────> │    COS      │
    ╰───────────╯                      ╱                 │             │
          │                           ╱                  ╰─────────────╯
          │                          ╱                            ╭─────────────────────────────────────────────────────╮
          ╰──╮                      ╱                             │{                                                    │
             │                     ╱                              │   "errno": 0,                                       │
             v                    ╱                               │   "pigeon": { "msgkey": "seeocr_output"},           │
       ╔══════════════════╗      ╱                                |   "ocr_res_json": "${COS_DOMAIN}/xxx/result.json"   │
       ║     CosFtpd      ║     ╱                                 │}                                                    │
       ║            ╭─────╨──────╮                                ╰─────────────────────────────────────────────────────╯
       ║            │ w  fifo  r │                                                 result
       ║            ╰─────╥──────╯                               ╭──────────────╮                ╭────────────────╮
       ╚══════════════════╝    │                                 │              │<───────────────│                │
                               │                            ╭───>│   Kafka      │                │    seeocr      │
                               │          ╭───────────╮     │    │              │───────────────>│                │
                               │ post     │           │─────╯    ╰──────────────╯                ╰────────────────╯
                               ╰────────> │  业务SQL   │                ^          seeocr_input
                                http      │           │                │
                                          ╰───────────╯                ╰──────────────────────────────────╮
                        /cosftpd/on_upload                                                                │
 _______________________________________________________________________________________________          │
/\                                                                                              \         │
\_| {                                                                                           |         │
  |     "ftp_user":"holyhand",                                                                  |         │
  |     "mac":"008e9b61ac6c",                                                                   |         │
  |     "cos_url":"${COS_DOMAIN}/holyhand/008e9b61ac6c/20221104/images/P22110420294710.jpg"     |         │
  | }                                                                                           |         │
  |   __________________________________________________________________________________________|_        │
   \_/____________________________________________________________________________________________/       │
                                                                                                          │
                _______________________________________________________________________________________________
               /\                                                                                              \
               \_|                                                                                             |
                 | {                                                                                           |
                 |     "cfg":{                                                                                 |
                 |         "pigeon":{                                                                          |
                 |             "msgkey":"seeocr_output"                                                        |
                 |         },                                                                                  |
                 |         "source":"${COS_DOMAIN}/holyhand/008e9b61ac6c/20221104/images/P22110420294710.jpg", |
                 |         "devmode": false,                                                                   |
                 |         "ocr_keys":[ "产品型号", "生产人数"],                                                  |
                 |         "det_max_side_len":800,                                                             |
                 |         "det_thresh":0.3,                                                                   |
                 |         "det_box_trhesh":0.6,                                                               |
                 |         "det_unclip_ratio":1.5,                                                             |
                 |         "rec_batch_num":6,                                                                  |
                 |         "rec_image_shape":[3, 48, 320],                                                     |
                 |         "process_code":""                                                                   |
                 |     }                                                                                       |
                 | }                                                                                           |
                 |   __________________________________________________________________________________________|_
                  \_/____________________________________________________________________________________________/

```


## 接口

### 摄像头抓拍图片接口 `/cosftpd/on_upload`

```json
{                                                                                                                                        
    "ftp_user":"holyhand",    // FTP用户名（可用来表示不同的工厂）                                                                                                            
    "mac":"008e9b61ac6c",     // 摄像头的MAC地址                                                                                                           
    "cos_url":"https://cosftpd-1301930378.cos.ap-beijing.myqcloud.com/holyhand/008e9b61ac6c/20221104/images/P22110420294710.jpg"  // 抓拍上传到COS的地址        
}                                                                                                                                        
```

### 执行OCR文字识别接口 `Kafka: seeocr_input`

1. 输入

```json
{
    "cfg":{
        "pigeon":{  // 服务器可以用pigeon携带自己的信息，算法返回信息里会携带pigeon信息（给服务器）。
            "msgkey":"seeocr_output"  // 算法执行结果返回结果到Kafka队列Topic:seeocr_output (默认）
        },
        "source":"https://cosftpd-1301930378.cos.ap-beijing.myqcloud.com/holyhand/008e9b61ac6c/20221104/images/P22110420294710.jpg", // 图片地址
        "devmode": false,  // 是否启动调试
        "ocr_keys":[ "产品型号", "生产人数"],  // 数组，指定需要识别的哪些文字
        "det_max_side_len":800,  // 正整数， 图片中文字最长边长
        "det_thresh":0.3,        // 浮点型，图片文字探测的判定阈值
        "det_box_trhesh":0.6,    // 浮点型，Box的判定阈值
        "det_unclip_ratio":1.5,  // 浮点型
        "rec_batch_num":6,       // 正整数
        "rec_image_shape":[3, 48, 320],  // 大小为3的数组（正整数）
        "process_code":""        // 字符串，插入的python 代码，纠正结果。
    }
}
```
-------------

2. 输出

```json
{
    "errno": 0,  // 结果码
    "pigeon": {
        "msgkey": "seeocr_output"
    },  // 返回服务器传来的pigeon
    "progress": 100,
    "ocr_res_json": "${COS_DOMAIN}/xxx/result.json" // ocr结果存放的地址
}
```
3. 结果result

```json
{
    "产品型号": "ABdhivm6KM",
    "生产人数": "3人",
    "线别": "2FhB",
    "标准节拍": "100S/PCS",
    "生产时间": "15时37分",
    "实际节拍": "0S/PCS",
    "计划产量": "1234PCS",
    "实际产量": "128PCS",
    "不良率": "36.7%",
    "完成率": "10.3%"
}
```